In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dtoolbioimage import ImageDataSet, Image3D, Image
import numpy as np

In [ ]:
imageds = ImageDataSet('azure://jicimagedatasets/cc9d757a-b149-4626-83be-abc5a3765cb8')

In [ ]:
venus_stack = imageds.get_stack('fca-3_FLC-Venus_root01', 'fca-3_FLC-Venus_root01 #1', 0)

In [ ]:
from dtoolbioimage.segment import Segmentation3D

In [ ]:
raw_segmentation = Segmentation3D.from_file('root01segmentation.tif')

In [ ]:
from dtoolbioimage.util.shape import spherality

In [ ]:
spherality_map = {l: spherality(raw_segmentation.region_in_bb(l)) for l in raw_segmentation.labels}

In [ ]:
spherality_map

In [ ]:
class SegmentationHeatMapPalette(object):
    
    def __init__(self, label_value_map):
        values = label_value_map.values()
        self.min_val = min(values)
        self.max_val = max(values)
                
        self.label_value_map = label_value_map
        
        assert self.max_val - self.min_val > 0
        
    def get_rgb_color(self, value):
        
        normalised_val = (value - self.min_val) / (self.max_val - self.min_val)
        
        g = int(normalised_val * 255)
        r = 255 - g
        b = g
        
        return r, g, b
    
    def __getitem__(self, key):
        
        default = [0, 0, 0]
        
        if key in self.label_value_map:
            value = self.label_value_map[key]
            return self.get_rgb_color(value)
        else:
            return default


In [ ]:
palette = SegmentationHeatMapPalette(spherality_map)

In [ ]:
palette.get_rgb_color(0.432)

In [ ]:
palette[-10]

In [ ]:
from dtoolbioimage.util.array import color_array    

In [ ]:
ca = color_array(raw_segmentation[:,:,10], palette)

In [ ]:
ca.view(Image)

In [ ]:
def filter_by_label_value_map(segmentation, label_value_map, threshold):
    result = segmentation.copy()
    
    for l, val in label_value_map.items():
        if val < threshold:
            result[np.where(segmentation == l)] = 0
            
    return result.view(Segmentation3D)

In [ ]:
filtered_segmentation = filter_by_label_value_map(raw_segmentation, spherality_map, 0.4)
filtered_segmentation

In [ ]:
from dtoolbioimage import zoom_to_match_scales

In [ ]:
zoomed_venus_stack = zoom_to_match_scales(venus_stack)

In [ ]:
def measure_by_label(segmentation, l):
    region_coords = np.where(segmentation==l)
    value = sum(zoomed_venus_stack[region_coords])
    size = len(region_coords[0])
    return value / size

In [ ]:
from scipy.ndimage.morphology import binary_erosion
def eroded_measure_by_label(segmentation, l):
    region_coords = np.where(segmentation==l)
    erodable = np.zeros(segmentation.shape, dtype=np.bool_)
    erodable[region_coords] = True
    eroded = binary_erosion(erodable)
    value = sum(zoomed_venus_stack[region_coords])
    size = len(region_coords[0])
    return value / size

In [ ]:
measure_by_label(filtered_segmentation, 5)

In [ ]:
def make_heatmap(segmentation):
    measures = {l: measure_by_label(segmentation, l) for l in segmentation.labels}
    
    palette = SegmentationHeatMapPalette(measures)
    
    return color_array(segmentation[:,:,10], palette).view(Image)

In [ ]:
make_heatmap(filtered_segmentation)

In [ ]:
zoomed_venus_stack[:,:,10]

In [ ]:
from dtoolbioimage.region import Region

In [ ]:
r = Region.select_from_array(filtered_segmentation, 5)

In [ ]:
r.view(Image3D)

In [ ]:
from scipy.ndimage.morphology import binary_erosion

In [ ]:
eroded = binary_erosion(r)

In [ ]:
eroded.view(Image3D)

In [ ]:
np.sum(r)

In [ ]:
np.sum(eroded)

In [ ]:
def erode_all(segmentation, iterations=3):
    result = np.zeros(segmentation.shape, dtype=np.uint32)
    
    for l in segmentation.labels:
        r = Region.select_from_array(segmentation, l)
        eroded = binary_erosion(r, iterations=iterations)
        result[np.where(eroded)] = l
        
    return result.view(Segmentation3D)

In [ ]:
eroded_segmentation = erode_all(filtered_segmentation)

In [ ]:
eroded_segmentation

In [ ]:
make_heatmap(eroded_segmentation)